In [1]:
import os,sys
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm
from sklearn.externals import joblib
import matplotlib.pyplot as plt
sys.path.append('../LIB/')
from env import ENV
from sklearn.preprocessing import normalize
from tqdm import tqdm

import os

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
import pickle

In [2]:
from keras.layers import Dense,Input,LSTM,Bidirectional,Activation,Conv1D,GRU,CuDNNGRU,Flatten,BatchNormalization
from keras.callbacks import Callback
from keras.layers import Dropout,Embedding,GlobalMaxPooling1D, MaxPooling1D, Add, Flatten
from keras.preprocessing import text, sequence
from keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate, SpatialDropout1D
from keras import initializers, regularizers, constraints, optimizers, layers, callbacks
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras.models import Model
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold

/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
def scan_nan_portion(df):
    portions = []
    columns = []
    for col in df.columns:
        columns.append(col)
        portions.append(np.sum(df[col].isnull())/len(df))
    return pd.Series(data=portions, index=columns)

In [4]:
X_Train = pd.read_pickle(ENV.application_train_cleaned.value)
print('Train shape: {}'.format(X_Train.shape))

X_Test = pd.read_pickle(ENV.application_test_cleaned.value)
print('Test shape: {}'.format(X_Test.shape))

X_pre = pd.read_pickle(ENV.previous_application_cleaned.value)
print('Previous App shape: {}'.format(X_pre.shape))

X_bu_b = pd.read_pickle(ENV.bureau_balance_clean.value)
print('Bureau Balance shape: {}'.format(X_bu_b.shape))

X_bu = pd.read_pickle(ENV.bureau_cleaned.value)
print('Bureau shape: {}'.format(X_bu.shape))

X_ins = pd.read_pickle(ENV.installments_payments_clean.value)
print('Installment shape: {}'.format(X_ins.shape))

X_pos = pd.read_pickle(ENV.POS_CASH_balance_clean.value)
print('POS CASH shape: {}'.format(X_pos.shape))

X_cc = pd.read_pickle(ENV.credit_card_balance_clean.value)
print('Credit Card shape: {}'.format(X_cc.shape))

Train shape: (307511, 122)
Test shape: (48744, 121)
Previous App shape: (1670214, 37)
Bureau Balance shape: (27299925, 3)
Bureau shape: (1716428, 17)
Installment shape: (13605401, 8)
POS CASH shape: (10001358, 8)
Credit Card shape: (3840312, 23)


In [5]:
X_ins = pd.read_pickle(ENV.installments_payments_clean.value)
print('Installment shape: {}'.format(X_ins.shape))

Installment shape: (13605401, 8)


In [6]:
sort_ins = X_ins.sort_values(['SK_ID_CURR','DAYS_INSTALMENT'])

# Process NUM_INSTALMENT_VERSION

In [7]:
col = 'NUM_INSTALMENT_VERSION'
mapping = pd.Series(index=range(0,179),data=range(0,179))
mapping[(mapping.index>=3)&(mapping.index<6)] = 3  
mapping[(mapping.index>=6)&(mapping.index<15)] = 4  
mapping[(mapping.index>=15)] = 5
sort_ins[col] = sort_ins[col].map(mapping)

# Process count install ment

In [8]:
groups = sort_ins.groupby('SK_ID_CURR')
dicts = {}
count = 0
mapping = sort_ins[['SK_ID_CURR','SK_ID_PREV']].drop_duplicates()
mapping = mapping.SK_ID_CURR.value_counts()
sort_ins['count_apps'] = sort_ins.SK_ID_CURR.map(mapping)
mapping = pd.Series(index=range(0,26),data=range(0,26))
mapping[(mapping.index>=7)&(mapping.index<15)] = 7  
mapping[(mapping.index>=15)&(mapping.index<29)] = 8
sort_ins['count_apps'] = sort_ins.count_apps.map(mapping)

# Process portion

In [9]:
sort_ins['payment_portion'] = sort_ins['AMT_PAYMENT'] / sort_ins['AMT_INSTALMENT'] 

In [10]:
sort_ins.loc[sort_ins['payment_portion']>1,'payment_portion'] = 2
sort_ins.loc[sort_ins['payment_portion']==1,'payment_portion'] = 1
sort_ins.loc[(sort_ins['payment_portion']>0.5)&(sort_ins['payment_portion']<1),'payment_portion'] = -1
sort_ins.loc[(sort_ins['payment_portion']>0.0)&(sort_ins['payment_portion']<=0.5),'payment_portion'] = -2
sort_ins.loc[sort_ins['payment_portion']==0,'payment_portion'] = -3
sort_ins['payment_portion'] = sort_ins['payment_portion'].fillna(0)

# Process total portion

In [11]:
groups = sort_ins.groupby('SK_ID_CURR')
ap = groups.AMT_PAYMENT.sum()
ai = groups.AMT_INSTALMENT.sum()
tportion = ap / ai
sort_ins['total_payment_portion'] = sort_ins.SK_ID_CURR.map(tportion)

In [12]:
sort_ins.loc[sort_ins['total_payment_portion']>1,'total_payment_portion'] = 2
sort_ins.loc[sort_ins['total_payment_portion']==1,'total_payment_portion'] = 1
sort_ins.loc[(sort_ins['total_payment_portion']>0.5)&(sort_ins['total_payment_portion']<1),'total_payment_portion'] = -1
sort_ins.loc[(sort_ins['total_payment_portion']>0.0)&(sort_ins['total_payment_portion']<=0.5),'total_payment_portion'] = -2
sort_ins.loc[sort_ins['total_payment_portion']==0,'total_payment_portion'] = -3
sort_ins['total_payment_portion'] = sort_ins['total_payment_portion'].fillna(0)

In [13]:
feature = list(sort_ins.columns)
feature.remove('SK_ID_PREV')
feature.remove('SK_ID_CURR')
feature.remove('DAYS_INSTALMENT')
feature.remove('DAYS_ENTRY_PAYMENT')
feature.remove('NUM_INSTALMENT_NUMBER')
feature.remove('AMT_INSTALMENT')
feature.remove('AMT_PAYMENT')

In [14]:
def get_group(df,cols):
    groups = df[cols[0]].astype('str')
    for each in cols[1:]:
        groups += '_' + df[each].astype('str')
    return groups

In [19]:
# sort_ins['words'] = get_group(sort_ins,feature)
sort_ins['words'] = sort_ins.index.astype('str')

In [16]:
def get_embeddings_index(sorted_df,nor_ebd):
    embeddings_index={}
    words_values = sorted_df['words'].values
    for index in range(len(words_values)):
        embeddings_index  [words_values[index]] = nor_ebd[index,:]
    return embeddings_index

def create_document(sorted_df):
    #Create document
    ids = sorted_df.SK_ID_CURR.values
    words = sorted_df.words.values
    document_dicts = {}

    id_list = []
    document_list = []

    for index in range(len(ids)) :
        if document_dicts.get(ids[index]) is None:
            document_dicts[ids[index]] = []
        document_dicts[ids[index]].append(words[index])

    for key in document_dicts :
        document_dicts[key] = ' '.join(document_dicts[key])
        id_list.append(key)
        document_list.append(document_dicts[key])


    df_doc = pd.DataFrame({'SK_ID_CURR':id_list, 'text':document_list})  
    df_doc_mapping  = df_doc.set_index('SK_ID_CURR').text

    train = X_Train[['SK_ID_CURR','TARGET']].copy()
    test = X_Test[['SK_ID_CURR']].copy()
    train['text'] = train.SK_ID_CURR.map(df_doc_mapping).fillna('notfound')
    test['text'] = test.SK_ID_CURR.map(df_doc_mapping).fillna('notfound')
    return train,test


def get_train_ebdMat(train,test,embeddings_index):
    X_train = train["text"].str.lower()
    X_test = test["text"].str.lower()
    y_train = train["TARGET"].values
    tok=text.Tokenizer(num_words=max_features,lower=True,filters='')
    tok.fit_on_texts(list(X_train)+list(X_test))
    X_train=tok.texts_to_sequences(X_train)
    X_test=tok.texts_to_sequences(X_test)
    x_train=sequence.pad_sequences(X_train,maxlen=maxlen)
    x_test=sequence.pad_sequences(X_test,maxlen=maxlen)
    print('...get ebd mat')
    word_index = tok.word_index
    #prepare embedding matrix
    num_words = min(max_features, len(word_index) + 1)
    print('num of words: {}'.format(num_words))
    embedding_matrix = np.zeros((num_words, embed_size))
    print(embedding_matrix.shape)
    for word, i in word_index.items():
        if i >= max_features:
            continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            # words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector
    return x_train,x_test,y_train,embedding_matrix,num_words

class_ratio =  sum(X_Train.TARGET ==0)/sum(X_Train.TARGET ==1)
class_ratio =  1
def get_rnn_model(num_words,embed_size,embedding_matrix):
    sequence_input = Input(shape=(maxlen, ))
    
    x = Embedding(num_words, embed_size, weights=[embedding_matrix],trainable = True)(sequence_input)
    x = BatchNormalization()(x)
    x = SpatialDropout1D(0.2)(x)
#     x = Bidirectional(GRU(16, return_sequences=True,dropout=0.1,recurrent_dropout=0.1))(x)
    x = Bidirectional(CuDNNGRU(8, return_sequences=True))(x)
    x = Conv1D(32, kernel_size = 3, padding = "valid", kernel_initializer = "glorot_uniform")(x)
    x = BatchNormalization()(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    x = concatenate([avg_pool, max_pool]) 
   
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(32, activation='relu')(x)
    x = Dropout(0.2)(x)
    preds = Dense(1, activation="sigmoid")(x)
    model = Model(sequence_input, preds)
#     Adam,RMSprop,Adagrad,Adadelta,Adamax,Nadam
#     model.compile(loss='binary_crossentropy',optimizer=Adam(lr=1e-4),metrics=['accuracy'])
    model.compile(loss='binary_crossentropy',optimizer=Adam(),metrics=['accuracy'])
    return model

def train_each_epoch(x,y,batch_size,model):
    model.fit(x, y, 
              batch_size=batch_size, 
              epochs=1,
              verbose=1,
              class_weight={0:1,1:class_ratio})
    return model

def load_model(model,filepath):
    model.load_weights(filepath)
    return model

def save_model(model, filepath):
    model.save_weights(filepath)

def train_each_fold(x,y,x_val,y_val,model,filepath,reportpath,predspath,
                    batch_size=512,total_epoch=40,patience=5,saving=True):
    ROC_AUC_SCORE = []
    for epoch in range(total_epoch):  
        model = train_each_epoch(x,y,batch_size,model)
        y_pred = model.predict(x_val,batch_size=3000,verbose=1)
        print(y_pred)
        score = roc_auc_score(y_val,y_pred)
        if len(ROC_AUC_SCORE) == 0:
            if saving:
                save_model(model,filepath)
            best_score = 0 
            if saving:
                print('saving preds...')
                pickle.dump(y_pred,open(predspath,'wb'))
        else:
            best_score = max(ROC_AUC_SCORE)
            if score >= best_score:
                if saving:
                    print('saving model to... {}'.format(filepath))
                    save_model(model,filepath)
                    print('saving preds...')
                    pickle.dump(y_pred,open(predspath,'wb'))
        ROC_AUC_SCORE.append(score)
        if saving:
            print('saving report to... {}'.format(reportpath))
            pickle.dump(ROC_AUC_SCORE,open(reportpath,'wb'))
        print('======= current {} / {}'.format(epoch,total_epoch))
        print('previous best roc is {}'.format(best_score))
        print('current roc is {}'.format(score))
        try:
            best_round = ROC_AUC_SCORE.index(best_score)
        except ValueError:
            best_round = -1
        if len(ROC_AUC_SCORE) > patience + best_round:
            print('reach patience! end loop')
            break
            
def train_5_folds(model_file,report_file,pred_file,pred_test_file,batch_size=512,total_epoch=400,patience=30,saving=False):
    train_fold_index = pickle.load(open(ENV.train_fold_index.value,'rb'))
    val_fold_index = pickle.load(open(ENV.val_fold_index.value,'rb'))

    for fold in range(len(train_fold_index)):
        print('!!!!!!!! Begin fold: {}'.format(fold))
        train_index = train_fold_index[fold]
        val_index = val_fold_index[fold]
        X_tra = x_train[train_index]
        y_tra = y_train[train_index]
        X_val = x_train[val_index]
        y_val = y_train[val_index]
        print('preparing train/val done!')
        print('before evaluating: {}'.format(model_file))
        model_file_evl = model_file.format(fold)
        report_file_evl = report_file.format(fold)
        pred_file_evl = pred_file.format(fold)
        pred_test_file_evl = pred_test_file.format(fold)
        model = get_rnn_model(num_words,embed_size,embedding_matrix)
        train_each_fold(X_tra, y_tra, X_val, y_val,
                        model,
                        filepath=model_file_evl,reportpath=report_file_evl,predspath=pred_file_evl,
                        batch_size=batch_size,total_epoch=total_epoch,patience=patience,saving=saving)
        #### predict test
        model = load_model(model,model_file_evl)
        test_preds = model.predict(x_test,batch_size=1500,verbose=1)
        pickle.dump(test_preds,open(pred_test_file_evl,'wb'))
        print('\n')


# Installment

In [20]:
label_mapping = X_Train.set_index('SK_ID_CURR').TARGET
test_mapping = pd.Series(index=X_Test.SK_ID_CURR, data=1)
sorted_df = sort_ins
#previous application
#13605401
max_features = 1000
#372
maxlen = 60
embed_size = 4

ebd = sorted_df[feature].values
#normalize
print('start normalize')
# nor_ebd = normalize(ebd, norm='max',axis=0)
nor_ebd = ebd
print('get embedding')
embeddings_index = get_embeddings_index(sorted_df,nor_ebd)
print('create document')
train,test = create_document(sorted_df)
print('get embedding Mat')
x_train,x_test,y_train,embedding_matrix,num_words = get_train_ebdMat(train,test,embeddings_index)
model_file = ENV.installment_rnn.value
report_file = ENV.installment_report.value
pred_file = ENV.installment_preds.value
pred_test_file = ENV.installment_preds_test.value
train_5_folds(model_file,report_file,pred_file,pred_test_file,batch_size=2000,total_epoch=500,patience=30,saving=False)

start normalize
get embedding
create document
get embedding Mat
...get ebd mat
num of words: 1000
(1000, 4)
!!!!!!!! Begin fold: 0
preparing train/val done!
before evaluating: ../LIB/../../data/rnn/installment/fold_{}.hdf5
Epoch 1/1


ResourceExhaustedError: OOM when allocating tensor with shape[2000,60,4]
	 [[Node: embedding_1/Gather = Gather[Tindices=DT_INT32, Tparams=DT_FLOAT, validate_indices=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](embedding_1/embeddings/read, embedding_1/Cast)]]
	 [[Node: metrics/acc/Mean_1/_249 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_2650_metrics/acc/Mean_1", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'embedding_1/Gather', defined at:
  File "/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages/ipykernel_launcher.py", line 

In [ ]:
train_5_folds(model_file,report_file,pred_file,pred_test_file,batch_size=2000,total_epoch=500,patience=30,saving=False)

In [ ]:
x

In [ ]:
X_train = train["text"].str.lower()
X_test = test["text"].str.lower()
y_train = train["TARGET"].values
tok=text.Tokenizer(num_words=max_features,lower=True,filters=None)
tok.fit_on_texts(list(X_train.values)+list(X_test.values))
X_train=tok.texts_to_sequences(X_train)
X_test=tok.texts_to_sequences(X_test)
x_train=sequence.pad_sequences(X_train,maxlen=maxlen)
x_test=sequence.pad_sequences(X_test,maxlen=maxlen)
print('...get ebd mat')
word_index = tok.word_index
#prepare embedding matrix
num_words = min(max_features, len(word_index) + 1)
embedding_matrix = np.zeros((num_words, embed_size))
for word, i in word_index.items():
    if i >= max_features:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
return x_train,x_test,y_train,embedding_matrix,num_words,tok

In [ ]:


label_mapping = X_Train.set_index('SK_ID_CURR').TARGET
test_mapping = pd.Series(index=X_Test.SK_ID_CURR, data=1)

#previous application
#13605401
max_features = 13605402
#372
maxlen = 60
embed_size = 6

# sorted_df = X_ins.sort_values(['SK_ID_CURR','DAYS_INSTALMENT']).copy()
# sorted_df['words'] = sorted_df.index.astype(str)
# feature = list(sorted_df.columns)
# feature.remove('SK_ID_PREV')
# feature.remove('SK_ID_CURR')
feature.remove('words')
ebd = sorted_df[feature].values
#normalize
print('start normalize')
# nor_ebd = normalize(ebd, norm='max',axis=0)
nor_ebd = ebd
print('get embedding')
embeddings_index = get_embeddings_index(sorted_df,nor_ebd)
print('create document')
train,test = create_document(sorted_df)
print('get embedding Mat')
x_train,x_test,y_train,embedding_matrix,num_words = get_train_ebdMat(train,test,embeddings_index)
model_file = ENV.installment_rnn.value
report_file = ENV.installment_report.value
pred_file = ENV.installment_preds.value
pred_test_file = ENV.installment_preds_test.value
train_5_folds(model_file,report_file,pred_file,pred_test_file,batch_size=2000,total_epoch=500,patience=30)

In [ ]:
a = train.text.apply(lambda x: len(x.split(' ')))
a.describe()

# POS_CASH

In [ ]:


label_mapping = X_Train.set_index('SK_ID_CURR').TARGET
test_mapping = pd.Series(index=X_Test.SK_ID_CURR, data=1)

#previous application
#10001358
max_features = 10001359
#295
maxlen = 300
embed_size = 6

sorted_df = X_pos.sort_values(['SK_ID_CURR','MONTHS_BALANCE'])
sorted_df['words'] = sorted_df.index.astype(str)
feature = list(sorted_df.columns)
feature.remove('SK_ID_PREV')
feature.remove('SK_ID_CURR')
feature.remove('words')
ebd = sorted_df[feature].values
#normalize
nor_ebd = normalize(ebd, norm='max',axis=0)
embeddings_index = get_embeddings_index(sorted_df,nor_ebd)
train,test = create_document(sorted_df)
x_train,x_test,y_train,embedding_matrix,num_words = get_train_ebdMat(train,test,embeddings_index)
model_file = ENV.POS_CASH_rnn.value
report_file = ENV.POS_CASH_report.value
pred_file = ENV.POS_CASH_preds.value
pred_test_file = ENV.POS_CASH_preds_test.value
train_5_folds(model_file,report_file,pred_file,pred_test_file)

# Credit Card

In [ ]:


label_mapping = X_Train.set_index('SK_ID_CURR').TARGET
test_mapping = pd.Series(index=X_Test.SK_ID_CURR, data=1)

#previous application
#10001358
max_features = 10001359
#192
maxlen = 193
embed_size = 21

sorted_df = X_cc.sort_values(['SK_ID_CURR','MONTHS_BALANCE'])
sorted_df['words'] = sorted_df.index.astype(str)
feature = list(sorted_df.columns)
feature.remove('SK_ID_PREV')
feature.remove('SK_ID_CURR')
feature.remove('words')
ebd = sorted_df[feature].values
#normalize
nor_ebd = normalize(ebd, norm='max',axis=0)
embeddings_index = get_embeddings_index(sorted_df,nor_ebd)
train,test = create_document(sorted_df)
x_train,x_test,y_train,embedding_matrix,num_words = get_train_ebdMat(train,test,embeddings_index)
model_file = ENV.credit_card_rnn.value
report_file = ENV.credit_card_report.value
pred_file = ENV.credit_card_preds.value
pred_test_file = ENV.credit_card_preds_test.value
train_5_folds(model_file,report_file,pred_file,pred_test_file)

In [ ]:
len(sorted_df['words'].unique())

In [ ]:
len(feature)

In [ ]:
max(train.text.apply(lambda x:len(x.split(' '))))

In [ ]:
max(test.text.apply(lambda x:len(x.split(' '))))

In [ ]:
train

# sort by sk curr and decision day

In [ ]:
sorted_pre = X_pre.sort_values(['SK_ID_CURR','DAYS_DECISION'])

# Use SK_ID_PREV as words

In [ ]:
sorted_pre['words'] = sorted_pre.SK_ID_PREV.astype(str)

# Get feature cols

In [ ]:
feature = list(sorted_pre.columns)
feature.remove('SK_ID_PREV')
feature.remove('SK_ID_CURR')
feature.remove('words')

# Get EBD

In [ ]:
ebd = sorted_pre[feature].values

# Nomralize

In [ ]:
nor_ebd = normalize(ebd, norm='max',axis=0)

# Get word EBD

In [ ]:
embeddings_index   = {}
words_values = sorted_pre['words'].values
for index in range(len(words_values)):
    embeddings_index  [words_values[index]] = nor_ebd[index,:]

# Train 5 folds

In [ ]:
def train_5_folds(model_file,report_file,pred_file,pred_test_file):
    train_fold_index = pickle.load(open(ENV.train_fold_index.value,'rb'))
    val_fold_index = pickle.load(open(ENV.val_fold_index.value,'rb'))

    for fold in range(len(train_fold_index)):
        print('!!!!!!!! Begin fold: {}'.format(fold))
        train_index = train_fold_index[fold]
        val_index = val_fold_index[fold]
        X_tra = x_train[train_index]
        y_tra = y_train[train_index]
        X_val = x_train[val_index]
        y_val = y_train[val_index]
        print('preparing train/val done!')
        model = get_rnn_model(num_words,embed_size,embedding_matrix)
        train_each_fold(X_tra, y_tra, X_val, y_val,
                        model,
                        filepath=model_file,reportpath=report_file,predspath=pred_file,
                        batch_size=512,total_epoch=40)
        #### predict test
        model = load_model(model,model_file)
        test_preds = model.predict(x_test,batch_size=5120,verbose=1)
        pickle.dump(test_preds,open(pred_test_file,'wb'))
        print('\n')

In [ ]:
train_fold_index = pickle.load(open(ENV.train_fold_index.value,'rb'))
val_fold_index = pickle.load(open(ENV.val_fold_index.value,'rb'))

for fold in range(len(train_fold_index)):
    print('!!!!!!!! Begin fold: {}'.format(fold))
    train_index = train_fold_index[fold]
    val_index = val_fold_index[fold]
    X_tra = x_train[train_index]
    y_tra = y_train[train_index]
    X_val = x_train[val_index]
    y_val = y_train[val_index]
    print('preparing train/val done!')
    model_file = ENV.previous_application_rnn.value.format(fold)
    report_file = ENV.previous_application_report.value.format(fold)
    pred_file = ENV.previous_application_preds.value.format(fold)
    pred_test_file = ENV.previous_application_preds_test.value.format(fold)
    model = get_rnn_model(num_words,embed_size,embedding_matrix)
    train_each_fold(X_tra, y_tra, X_val, y_val,
                    model,
                    filepath=model_file,reportpath=report_file,predspath=pred_file,
                    batch_size=512,total_epoch=40)
    #### predict test
    model = load_model(model,model_file)
    test_preds = model.predict(x_test,batch_size=5120,verbose=1)
    pickle.dump(test_preds,open(pred_test_file,'wb'))
    print('\n')

In [ ]:
ssss

In [ ]:
sequence_input = Input(shape=(maxlen, ))
x = Embedding(num_words, embed_size, weights=[embedding_matrix],trainable = False)(sequence_input)
x = SpatialDropout1D(0.2)(x)
x = Bidirectional(GRU(128, return_sequences=True,dropout=0.1,recurrent_dropout=0.1))(x)
x = Conv1D(64, kernel_size = 1, padding = "valid", kernel_initializer = "glorot_uniform")(x)
avg_pool = GlobalAveragePooling1D()(x)
max_pool = GlobalMaxPooling1D()(x)
x = concatenate([avg_pool, max_pool]) 
x = Dense(128, activation='relu')(x)
x = Dropout(0.1)(x)
preds = Dense(1, activation="sigmoid")(x)
model = Model(sequence_input, preds)
model.compile(loss='binary_crossentropy',optimizer=Adam(lr=1e-3),metrics=['accuracy'])

In [ ]:
batch_size = 512
epochs = 40
X_tra, X_val, y_tra, y_val = train_test_split(x_train, y_train, train_size=0.9, random_state=233)

In [ ]:
# filepath="../input/best-model/best.hdf5"
filepath="weights_base.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='max')
early = EarlyStopping(monitor="val_loss", mode="max", patience=40)
ra_val = RocAucEvaluation(validation_data=(X_val, y_val), interval = 1)
callbacks_list = [ra_val,checkpoint, early]

## Train

In [ ]:
model.save_weights("weights_base.best123.hdf5")

In [ ]:
model.load_weights("weights_base.best123.hdf5")

In [ ]:

preds = model.predict(X_val,batch_size=5120,verbose=1)

In [ ]:
model.fit(X_tra, y_tra, batch_size=batch_size, epochs=epochs, validation_data=(X_val, y_val),callbacks = callbacks_list,verbose=1,class_weight={0:1,1:11.5})